In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import qmc
from scipy.stats import norm

Grundsätzliche Parameter festlegen

In [2]:
T = 1
d = 2
delta_t = T/d

S_0 = 100
mu = 0.045
sigma = 0.3


Standard Brownian Motion Simulation

In [9]:
def create_standard_bm(d, delta_t):
#create lower trinangular matrix with ones
    lower_tri_matr = np.tril(np.ones([d,d]))
    A = math.sqrt(delta_t)*lower_tri_matr
    #create Brownian motion
    w = A.dot(np.random.normal(0,1,d))

    #display(A)
    #display(w)
    

    return w
    plt.plot(w)
    plt.show()


Asset

In [4]:
def create_asset_prices(d, delta_t, S_0, mu, sigma):
    S = np.zeros(d+1) #+1 weil d mal simuliert wird und aber der Startpreis des Assets auch in dem Vektor steht
    S[0] = S_0
    delta_S = []

    w = create_standard_bm(d, delta_t)

    #erstelle gesamten Vekjtor w mit w_0
    delta_w = np.concatenate((w[0],np.diff(w)),axis=None)
    
    for i in range(d):
        delta_S.append(mu*S[i]*delta_t + sigma*S[i]*delta_w[i])
        S[i+1] = S[i] + delta_S[i]
    '''
    for i in range(d):
        delta_S.append(mu*S[i]*delta_t + sigma*S[i]*math.sqrt(delta_t)*np.random.normal(0,1,1))
        S[i+1] = S[i] + delta_S[i]
    '''
    return S
    
    '''
    plt.plot(s)
    plt.title("Asset prices for mu = " + str(mu) + ", S_0 = " + str(s_0) + ", Sigma = " + str(sigma))
    plt.ylabel("Asset prices")
    plt.show()
    '''


## Alternative Version of creating asset prices as comparison test

In [10]:
def create_asset_prices_directly(d, delta_t, S_0, mu, sigma):
    S = np.zeros(d+1) #+1 weil d mal simuliert wird und aber der Startpreis des Assets auch in dem Vektor steht
    S[0] = S_0

    w = create_standard_bm(d, delta_t) 
   
    for i in range(d):
        S[i+1] = S[0]*np.exp(sigma*w[i] + (mu-(sigma**2)/2)*(i+1)*delta_t)
    return S
    

Test, ob Asset prices richtig modelliert sind. Vergleich mit exakter Berechnung ergibt, dass Ergebnis stimmt.

In [8]:
number_paths = 10
mc_vector = np.empty(number_paths)
for i in range(number_paths):
    mc_vector[i] = create_asset_prices(d, delta_t, S_0, mu, sigma)[2]
    
display(np.average(mc_vector))

display(S_0*math.exp(mu*d*delta_t))

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
test_S = display(create_asset_prices(d, delta_t, S_0, mu, sigma))

array([100.        , 112.21881902])

In [7]:
#WICHTIG: S_0 muss im Vektor S sein!!! (per Definition)
def ratched_option(d, delta_t, S_0, mu, sigma): 
    #S = create_asset_prices(d, delta_t, S_0, mu, sigma)

    S = create_asset_prices_directly(d, delta_t, S_0, mu, sigma) #nur zum Testen

    #display(S)
    #indic = np.array([1 if s > 0 else 0 for s in np.diff(S)])
    indic = np.diff(S)>0
    #display(indic)
    #display(S[1:S.size])
    payoff = np.average(indic*S[1:S.size])  #python spezifisch fehlt dann das erste Element
    #display(payoff)
    #display(S)
    #display(S[1:(S.size)])
    #display(payoff)
    return payoff

In [9]:
ratched_option(d, delta_t, S_0, mu, sigma)

58.83763911055531

# Monte Carlo Simulation

Exaktes Ergebnis

In [12]:
Exp = (S_0/d)*norm.cdf((mu-((sigma**2)/2))*math.sqrt(delta_t)/sigma+sigma*math.sqrt(delta_t))*np.sum(np.exp(mu*delta_t*(np.array(range(d))+1)))
display(Exp)

60.40825215088241

Monte Carlo Simulation

In [11]:
number_paths = 100000
mc_vector = np.empty(number_paths)
for i in range(number_paths):
    mc_vector[i] = ratched_option(d, delta_t, S_0, mu, sigma) 

np.average(mc_vector)

60.58063396505514

In [ ]:
np.exp(-1)*71.28001584076893

26.22245239419363